In [2]:
from tools import get_puzzle, show_problem_1, show_problem_2

TODAY = 5
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)

https://adventofcode.com/2023/day/5
## --- Day 5: If You Give A Seed A Fertilizer ---


You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.


"A water source? Island Island **is** the water source!" You point out that Snow Island isn't receiving any water.


"Oh, we had to stop the water because we **ran out of sand** to [filter](https://en.wikipedia.org/wiki/Sand_filter) it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.


"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"


You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our **food production problem** . The latest Island Island [Almanac](https://en.wikipedia.org/wiki/Almanac) just arrived and we're having trouble making sense of it."


The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil123``and fertilizer123``aren't necessarily related to each other.


For example:


```
 seeds: 79 14 55 13
 
 seed-to-soil map:
 50 98 2
 52 50 48
 
 soil-to-fertilizer map:
 0 15 37
 37 52 2
 39 0 15
 
 fertilizer-to-water map:
 49 53 8
 0 11 42
 42 0 7
 57 7 4
 
 water-to-light map:
 88 18 7
 18 25 70
 
 light-to-temperature map:
 45 77 23
 81 45 19
 68 64 13
 
 temperature-to-humidity map:
 0 69 1
 1 0 69
 
 humidity-to-location map:
 60 56 37
 56 93 4

```


The almanac starts by listing which seeds need to be planted: seeds79``,14``,55``, and13``.


The rest of the almanac contains a list of **maps** which describe how to convert numbers from a **source category** into numbers in a **destination category** . That is, the section that starts withseed-to-soil map:``describes how to convert a **seed number** (the source) to a **soil number** (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.


Rather than list every source number and its corresponding destination number one by one, the maps describe entire **ranges** of numbers that can be converted. Each line within a map containsthree numbers: the **destination range start** , the **source range start** , and the **range length** .


Consider again the exampleseed-to-soil map``:


```
 50 98 2
 52 50 48

```


The first line has a **destination range start** of50``, a **source range start** of98``, and a **range length** of2``. This line means that the source range starts at98``and contains two values:98``and99``. The destination range is the same length, but it starts at50``, so its two values are50``and51``. With this information, you know that seed number98``corresponds to soil number50``and that seed number99``corresponds to soil number51``.


The second line means that the source range starts at50``and contains48``values:50``,51``, ...,96``,97``. This corresponds to a destination range starting at52``and also containing48``values:52``,53``, ...,98``,99``. So, seed number53``corresponds to soil number55``.


Any source numbers that **aren't mapped** correspond to the **same** destination number. So, seed number10``corresponds to soil number10``.


So, the entire list of seed numbers and their corresponding soil numbers looks like this:


```
 seed  soil
 0     0
 1     1
 ...   ...
 48    48
 49    49
 50    52
 51    53
 ...   ...
 96    98
 97    99
 98    50
 99    51

```


With this map, you can look up the soil number required for each initial seed number:


+ Seed number79``corresponds to soil number81``.
+ Seed number14``corresponds to soil number14``.
+ Seed number55``corresponds to soil number57``.
+ Seed number13``corresponds to soil number13``.


The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find **the lowest location number that corresponds to any of the initial seeds** . To do this, you'll need to convert each seed number through other categories until you can find its corresponding **location number** . In this example, the corresponding types are:


+ Seed79``, soil81``, fertilizer81``, water81``, light74``, temperature78``, humidity78``, **location82``** .
+ Seed14``, soil14``, fertilizer53``, water49``, light42``, temperature42``, humidity43``, **location43``** .
+ Seed55``, soil57``, fertilizer57``, water53``, light46``, temperature82``, humidity82``, **location86``** .
+ Seed13``, soil13``, fertilizer52``, water41``, light34``, temperature34``, humidity35``, **location35``** .


So, the lowest location number in this example is35``.


 **What is the lowest location number that corresponds to any of the initial seed numbers?** 




In [ ]:
def parse_data(data):
    new_data = []
    for row, card in enumerate(data):
        new_data.append([row+1, 
                         set( map (int, card.split(":")[1].split("|")[0].split()) ),
                        set( map (int, card.split(":")[1].split("|")[1].split()) )])
    return new_data

def calculate_prize(card):
    prize = 0
    for num in card[2]:
        if num in card[1]:
            if prize == 0:
                prize = 1
            else:
                prize *= 2
    return prize
    
def solution_1(cards):
    cards = parse_data(cards)
    prizes = []
    for card in cards:
        prizes.append(calculate_prize(card))
    return sum(prizes)

#print(solution_1 (puzzle.test))
assert solution_1 (puzzle.test) ==13
print( f"Solution 1:  {solution_1 (puzzle.data)} is the colorful cards total of points")

In [1]:
print("\n".join("|".join(puzzle.test).split(":")))

NameError: name 'puzzle' is not defined